In [1]:
import math
import pandas

def month_diff(a, b):
    return 12 * (a.year - b.year) + (a.month - b.month)

def elapsed_months(record):
    return month_diff(valdate, record.StartDate)

def start_age_months(record):
    return month_diff(record.StartDate, record.BirthDate)


In [2]:
valdate = pandas.Timestamp('2020-10-01')

In [3]:
contracts = pandas.read_excel('SampleInput.xlsx')
contracts = contracts.set_index('ContractNo')
contracts['ElapsedMonths'] = contracts.apply(elapsed_months, 'columns')
contracts['StartAgeMonths'] = contracts.apply(start_age_months, 'columns')
contracts

,Product,StartDate,TermMonths,BirthDate,Sex,SumInsured,ElapsedMonths,StartAgeMonths
ContractNo,,,,,,,,
CTR4983,TermFix,2008-12-01,255,1976-04-01,M,40000,142,392
CTR5328,TermFix,2016-04-01,255,1960-09-01,M,330000,54,667
CTR2626,TermFix,2008-02-01,292,1985-10-01,M,180000,152,268
CTR2976,TermFix,1991-07-01,362,1969-04-01,M,370000,351,267
CTR1970,TermFix,1997-12-01,286,1981-11-01,M,310000,274,193
CTR5918,TermFix,1990-03-01,380,1966-05-01,M,130000,367,286
CTR7871,TermFix,1998-02-01,278,1981-12-01,M,130000,272,194
CTR3549,TermFix,1998-06-01,297,1969-06-01,F,480000,268,348
CTR5031,TermFix,2016-10-01,251,1997-03-01,F,100000,48,235


In [4]:
def project(record):
    global grec
    grec = record
    alpha = {'M': 1.7e-5, 'F': 1.5e-5}[record.Sex]
    beta = 0.0081
    survivors = 1
    accum_deaths = 0

    Time = [0]
    AgeMonths = [record.StartAgeMonths]
    Survivors = [survivors]
    AccumDeaths = [accum_deaths]
    Deaths = [None]

    for t in range(1, record.TermMonths - record.ElapsedMonths + 2):
        age_months = record.StartAgeMonths + t
        # use mortality from previous month
        mortality = min(1.0, alpha * math.exp((age_months-1) * beta))
        deaths = survivors * mortality
        survivors -= deaths
        accum_deaths += deaths

        Time.append(t)
        AgeMonths.append(age_months)
        Survivors.append(survivors)
        AccumDeaths.append(accum_deaths)
        Deaths.append(deaths)

    return pandas.DataFrame({
        'ContractNo': [record.name]*len(Time),
        'Time': Time,
        'Survivors': Survivors,
        'AccumDeaths': AccumDeaths,
        'Deaths': Deaths,
        }).set_index(['ContractNo', 'Time'])

In [5]:
projection = pandas.concat(contracts.apply(project, 'columns').tolist())
projection = projection.merge(contracts['SumInsured'], how='left', left_index=True, right_index=True)
projection['DeathClaim'] = projection['SumInsured'] * projection['Deaths']
projection

Survivors  AccumDeaths    Deaths  SumInsured   DeathClaim
ContractNo Time                                                           
CTR4983    0      1.000000     0.000000       NaN       40000          NaN
           1      0.999593     0.000407  0.000407       40000    16.273492
           2      0.999183     0.000817  0.000410       40000    16.399168
           3      0.998770     0.001230  0.000413       40000    16.525760
           4      0.998354     0.001646  0.000416       40000    16.653273
...                    ...          ...       ...         ...          ...
CTR5504    259    0.701129     0.298871  0.002295      440000  1009.628143
           260    0.698824     0.301176  0.002306      440000  1014.519091
           261    0.696507     0.303493  0.002317      440000  1019.406599
           262    0.694179     0.305821  0.002328      440000  1024.290165
           263    0.691840     0.308160  0.002339      440000  1029.169284

[2014 rows x 5 columns]

In [6]:
contracts['TotalDeathClaim'] = projection.groupby(['ContractNo'])[['DeathClaim']].sum()
contracts

,Product,StartDate,TermMonths,BirthDate,Sex,SumInsured,ElapsedMonths,StartAgeMonths,TotalDeathClaim
ContractNo,,,,,,,,,
CTR4983,TermFix,2008-12-01,255,1976-04-01,M,40000,142,392,2925.677565
CTR5328,TermFix,2016-04-01,255,1960-09-01,M,330000,54,667,282115.717584
CTR2626,TermFix,2008-02-01,292,1985-10-01,M,180000,152,268,6900.944389
CTR2976,TermFix,1991-07-01,362,1969-04-01,M,370000,351,267,685.849655
CTR1970,TermFix,1997-12-01,286,1981-11-01,M,310000,274,193,343.376504
CTR5918,TermFix,1990-03-01,380,1966-05-01,M,130000,367,286,330.514761
CTR7871,TermFix,1998-02-01,278,1981-12-01,M,130000,272,194,76.285459
CTR3549,TermFix,1998-06-01,297,1969-06-01,F,480000,268,348,4063.821560
CTR5031,TermFix,2016-10-01,251,1997-03-01,F,100000,48,235,5088.186381
